In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('house-price-tehran-iran/housePrice.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/house-price-tehran-iran/housePrice.csv'

In [1]:
df.sample(5)

NameError: name 'df' is not defined

In [ ]:
type(df)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Area'] = df['Area'].apply(lambda x: re.sub(',', '', x))
df["Area"] = pd.to_numeric(df["Area"] , errors='coerce')

In [2]:
df.info()

NameError: name 'df' is not defined

In [3]:
df

NameError: name 'df' is not defined

In [4]:
df.shape

NameError: name 'df' is not defined

In [5]:
print(df.dtypes)

NameError: name 'df' is not defined

In [6]:
df.isnull().sum()

NameError: name 'df' is not defined

In [7]:
df.dropna(inplace=True)

NameError: name 'df' is not defined

In [8]:
df.shape

NameError: name 'df' is not defined

In [9]:
df['Parking'].value_counts(normalize=True)*100

NameError: name 'df' is not defined

In [10]:
pd.crosstab( df.Parking, df.Room )

NameError: name 'pd' is not defined

In [11]:
below_85 = df[ df.Area <= 85 ]
len(below_85)

NameError: name 'df' is not defined

In [ ]:
df[ df.Area <= 85 ]["Parking"].value_counts( )